# Library

In [2]:
# Librerie standard
import os
import random
import time
import re
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torchvision.transforms import functional as TF

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
import torchvision.models as models
from sklearn.svm import SVC

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import shutil 

# Librerie per il parallelismo e il multiprocessing
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

# Librerie per la gestione dei dataset
from torch.utils.data import Dataset, DataLoader

# Librerie per modelli e trasformazioni in PyTorch
from torchvision import transforms

from collections import Counter
from sklearn.model_selection import train_test_split
import warnings

from torchvision import models
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch import optim
import torch

# Path

In [ ]:
#Output folders and file names
OUT_COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

coco_json_pth = in_dataset_pth / OUT_COCO_JSON_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

# PROPOSALS
OUT_PROPOSALS_FLDR_NM = 'proposals'
prop_fldr = Path(f'/kaggle/working/{OUT_PROPOSALS_FLDR_NM}')
PROP_COCO_JSON_NM = 'proposals.json'
proposals_json = out_dataset_pth / PROP_COCO_JSON_NM
ACTPROP_COCO_JSON_NM ='active_regions.json'
actproposals_json = out_dataset_pth / ACTPROP_COCO_JSON_NM

#DATASET
train_path = '/kaggle/working/train.json'
test_path = '/kaggle/working/test.json'
val_path = '/kaggle/working/val.json'

random.seed(RANDOM_SEED)

In [ ]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)
clean_output(prop_fldr)

In [ ]:
# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# Splitting

# DataLoader

In [ ]:
class CustomDataset(Dataset):
    def init(self, txt_file, img_dir, coco_json_file, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - txt_file: Il file di testo contenente i percorsi delle immagini.
        - img_dir: La cartella delle immagini.
        - coco_json_file: Il file JSON in formato COCO contenente le annotazioni.
        - aug: Booleano per attivare o meno l'augmentazione.
        """
        def generate_id(file_name):
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')
 
        with open(txt_file, 'r') as f:
            self.image_paths = [line.strip() for line in f.readlines()]
 
        self.img_dir = img_dir
 
        # Carica il file JSON delle annotazioni COCO
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)
 
        # Crea una struttura per le annotazioni
        self.image_annotations = {}
        self.image_bboxes = {}
 
        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # Formato COCO [x_min, y_min, width, height]
 
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []
 
            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(bbox)
 
        # Mappa per associare ID immagine a file_name
        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        }
 
        # Trasformazioni di base e di augmentation
        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),   
        ])
 
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ])
 
        self.aug = aug
 
    def len(self):
        return len(self.image_paths)
 
    def getitem(self, index):
        # Estrai il nome dell'immagine e l'ID corrispondente
        img_name = os.path.basename(self.image_paths[index])
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', ''))
        if img_id not in self.image_info:
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size
        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        # Estrai le annotazioni e i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])
        if not bboxes:  # Immagini senza annotazioni
            target = {
                "boxes": torch.zeros((0, 4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64)
            }
        else:
            # Converte da formato COCO [x_min, y_min, width, height] a [x_min, y_min, x_max, y_max]
            scale_x = 320 / original_width
            scale_y = 320 / original_height
            scaled_bboxes = [
                torch.tensor([  # [x1, y1, x2, y2]
                    bbox[0] * scale_x,               # x_min
                    bbox[1] * scale_y,               # y_min
                    (bbox[0] + bbox[2]) * scale_x,   # x_max
                    (bbox[1] + bbox[3]) * scale_y    # y_max
                ], dtype=torch.float32)
                for bbox in bboxes
            ]

In [ ]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)
    
    # Varia le dimensioni delle immagini per garantire che siano tutte della stessa dimensione
    # La dimensione di uscita deve essere la stessa per tutte le immagini del batch
    images = [F.resize(img, (320, 320)) for img in images]  # Assumendo che 320 sia la dimensione target

    # Restituisci il batch
    return images, list(targets)

In [ ]:
# Creazione dei dataset
train_dataset_frcc = CustomDataset(train_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=True)
valid_dataset_frcc = CustomDataset(val_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  
test_dataset_frcc = CustomDataset(test_txt_pth, save_images_fldr_pth, new_coco_json_pth, aug=False)  

# Creazione dei DataLoader
train_loader_frcc = DataLoader(train_dataset_frcc, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader_frcc = DataLoader(valid_dataset_frcc, batch_size=8, shuffle=False, collate_fn=collate_fn)
test_loader_frcc = DataLoader(test_dataset_frcc, batch_size=8, shuffle=False, collate_fn=collate_fn)

## Check DataLoader

In [ ]:
# Numero totale di campioni per ogni DataLoader
train_size = len(train_loader_frcc.dataset)
val_size = len(val_loader_frcc.dataset)
test_size = len(test_loader_frcc.dataset)

# Numero di batch per ogni DataLoader
train_batches = len(train_loader_frcc)
val_batches = len(val_loader_frcc)
test_batches = len(test_loader_frcc)

# Visualizza i risultati
print(f"Numero totale di elementi nel train_loader: {train_size}")
print(f"Numero totale di batch nel train_loader: {train_batches}")
print(f"Numero totale di elementi nel val_loader: {val_size}")
print(f"Numero totale di batch nel val_loader: {val_batches}")
print(f"Numero totale di elementi nel test_loader: {test_size}")
print(f"Numero totale di batch nel test_loader: {test_batches}")

# Somma totale degli elementi nei DataLoader
total_elements = train_size + val_size + test_size
print(f"Numero totale di elementi in tutti i DataLoader: {total_elements}")

# Modello Faster R-CNN (Resnet50)

In [ ]:
def train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, save_model=True):
    """
    Funzione per il training e la validazione del modello Faster R-CNN.
    
    Args:
    - model: il modello Faster R-CNN.
    - train_loader: DataLoader per il training set.
    - val_loader: DataLoader per il validation set.
    - optimizer: ottimizzatore per il modello.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    - num_epochs: numero di epoche (default: 10).
    - save_model: se salvare il modello dopo ogni epoca (default: True).
    
    Returns:
    - losses_per_epoch: lista delle perdite per epoca durante il training.
    """
    model.to(device)
    losses_per_epoch = []

    for epoch in range(num_epochs):
        print(f"\nEpoca {epoch + 1}/{num_epochs}")
        model.train()
        total_loss = 0

        # Training loop con tqdm
        train_loop = tqdm(train_loader, desc="Training", leave=False)
        for images, targets in train_loop:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Calcola le perdite
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            # Ottimizzazione
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()
            train_loop.set_postfix(loss=losses.item())

        losses_per_epoch.append(total_loss)
        print(f"Perdita Totale per l'epoca {epoch + 1}: {total_loss:.4f}")

        # Validazione
        model.eval()
        val_loop = tqdm(val_loader, desc="Validazione", leave=False)
        with torch.no_grad():
            for images, targets in val_loop:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                predictions = model(images)
                val_loop.set_postfix(processed=len(predictions))  # Placeholder per metriche future

        # Salva il modello
        if save_model:
            torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")
            print(f"Modello salvato: model_epoch_{epoch + 1}.pth")

    return losses_per_epoch

In [ ]:
def test_model(model, test_loader, device):
    """
    Funzione per il testing del modello Faster R-CNN.
    
    Args:
    - model: il modello Faster R-CNN.
    - test_loader: DataLoader per il test set.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    
    Returns:
    - predictions: lista delle predizioni per ogni batch.
    """
    model.to(device)
    model.eval()
    predictions = []

    print("\nInizio testing...")
    test_loop = tqdm(test_loader, desc="Testing", leave=False)
    with torch.no_grad():
        for images, _ in test_loop:  # Durante il test, i target possono essere ignorati
            images = [img.to(device) for img in images]
            preds = model(images)
            predictions.extend(preds)
            test_loop.set_postfix(processed=len(predictions))

    print("Testing completato.")
    return predictions

In [ ]:
# Carica il modello Faster R-CNN con ResNet50 e FPN
model = fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 12  # 11 classi + 1 per il background

# Modifica il numero di classi in output
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Imposta il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configurazione training
num_epochs = 10
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
# Training e validazione
losses_per_epoch = train_and_validate(
    model=model,
    train_loader=train_loader_frcc,
    val_loader=val_loader_frcc,
    optimizer=optimizer,
    device=device,
    num_epochs=num_epochs,
    save_model=True
)

In [ ]:
# Testing
test_predictions = test_model(
    model=model,
    test_loader=test_loader_frcc,
    device=device
)